In [76]:
import requests as req
from bs4 import BeautifulSoup as bs
from concurrent.futures import ThreadPoolExecutor

In [29]:
 # yahoo_stock_URL
url = "https://tw.stock.yahoo.com/quote/8070.TW"


# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 


In [56]:
# 嘗試找到股票名稱.價格.漲跌幅
title = soup.select('h1')[1]
price = soup.select_one('.Fz\(32px\)')
percent = soup.select('.Fz\(20px\)')[0]
status = ""
print(title)
print(price)
print(percent)


<h1 class="C($c-link-text) Fw(b) Fz(24px) Mend(8px)">長華*</h1>
<span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-up)">48.70</span>
<span class="Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-up)"><span class="Mend(4px) Bds(s)" style="border-color:transparent transparent #ff333a transparent;border-width:0 6.5px 9px 6.5px"></span>1.65</span>


In [72]:
# 如果 main-0-QuoteHeader-Proxy id 的 div 裡有 C($c-trend-down) 的 class
# 表示狀態為下跌
down = soup.select_one('.C\(\$c-trend-down\)')

# 如果 main-0-QuoteHeader-Proxy id 的 div 裡有 C($c-trend-up) 的 class
# 表示狀態為上漲
up = soup.select_one('.C\(\$c-trend-up\)')
print(up)
print(down)

<span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-up)">48.70</span>
<span class="Mstart(5px) C($c-trend-down)">5,971<span class="Fw(n)">(<!-- -->49.41%<!-- -->)</span></span>


In [75]:
if up:
    status = '+'
elif down:
    status = '-'
else:
    status = ''

# 印出結果
print(f'{title.get_text()} : {price.get_text()} ( {status}{percent.get_text()} )')  

長華* : 48.70 ( +1.65 )


In [90]:
#想抓取的股票網址清單
stock_urls = [
  'https://tw.stock.yahoo.com/quote/8070',
  'https://tw.stock.yahoo.com/quote/2352',
  'https://tw.stock.yahoo.com/quote/8150',
  'https://tw.stock.yahoo.com/quote/2344'
]

def getStock(stock_urls):
    res = req.get(stock_urls) 
    soup = bs(res.text, "lxml") 
    title = soup.select('h1')[1]
    price = soup.select_one('.Fz\(32px\)')
    percent = soup.select('.Fz\(20px\)')[0]
    status = ""
    try:
        if up:
          status = '+'
        elif down:
          status = '-'
    except:
        state = ''
    
    return f'{title.get_text()} : {price.get_text()} ( {status}{percent.get_text()} )'
      
# 使用執行緒池來並行抓取股票資訊
with ThreadPoolExecutor() as executor:
    results = list(executor.map(getStock, stock_urls))  # 收集所有結果

# 印出所有結果
for result in results:
    print(result)

長華* : 48.70 ( +1.65 )
佳世達 : 34.80 ( +0.05 )
南茂 : 37.55 ( +0.20 )
華邦電 : 23.95 ( +0.15 )
